In [3]:
from modelzipper.tutils import *
from peft import LoraConfig, get_peft_model, PeftModelForCausalLM
import transformers
from datasets import load_from_disk, load_dataset
import subprocess
from tqdm import trange
from pprint import pprint
import numpy as np

/opt/conda/envs/pan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-08-23 20:11:09


In [2]:
data = load_from_disk('/nvme/zecheng/data/iclr2025/llama3-80k-train-data/dpo_data/chunk_16_size_1024')

In [20]:
pprint(data['train'][0].keys())

print(len(data['train'][-4]['chosen_input_ids']))
print((np.array(data['train'][-4]['chosen_input_ids']) != 0).sum())
print(np.array(data['train'][-4]['chosen_attention_mask']).sum())
print((np.array(data['train'][-4]['chosen_labels']) != -100).sum())
chosen_attention_mask = np.array(data['train'][-4]['chosen_attention_mask']) != 0
chosen_position_ids = np.array(data['train'][-4]['chosen_position_ids'])
print(chosen_position_ids[chosen_attention_mask].shape)
masked_ids = chosen_position_ids[chosen_attention_mask]
print(np.all(masked_ids[:-1] <= masked_ids[1:]))


dict_keys(['chosen_input_ids', 'chosen_attention_mask', 'chosen_position_ids', 'chosen_labels', 'reject_1_input_ids', 'reject_1_attention_mask', 'reject_1_position_ids', 'reject_1_labels', 'reject_2_input_ids', 'reject_2_attention_mask', 'reject_2_position_ids', 'reject_2_labels'])
18405
16706
16708
270
(16708,)
True


### 合并Lora的ckpt

In [4]:
BASE_MODEL_PATH = "/data/zecheng/hf_models/Llama-3-8B-Instruct-80K-QLoRA-Merged"
PEFT_MODEL_PATH = "/data/zecheng/ckpt/lcao_v6/peft_paths"
SAVE_PATH = "/data/zecheng/ckpt/lcao_v6/merged_ckpt"

tokenizer = transformers.AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
base_model = transformers.AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH)
peft_model = PeftModelForCausalLM.from_pretrained(base_model, PEFT_MODEL_PATH)

peft_model = peft_model.merge_and_unload()
peft_model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.61it/s]


[2024-08-22 02:58:45,239] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-08-22 02:58:45,312] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


/data/anaconda3/envs/zecheng/compiler_compat/ld: cannot find -laio: 没有那个文件或目录
collect2: error: ld returned 1 exit status


('/data/zecheng/ckpt/lcao_v6/merged_ckpt/tokenizer_config.json',
 '/data/zecheng/ckpt/lcao_v6/merged_ckpt/special_tokens_map.json',
 '/data/zecheng/ckpt/lcao_v6/merged_ckpt/tokenizer.json')

In [1]:
import datasets

train_data = datasets.load_dataset("/data/zecheng/data/long_llm_data")

In [7]:
train_data['train'][0]

{'conversations': [{'content': 'Find the volume of a cube with side lengths of 6 cm. Output the answer in cubic centimeters.',
   'role': 'user'},
  {'content': '216 cm^3.', 'role': 'assistant'}]}

### 在python里面写rclone终端命令，方便快捷传输数据

In [6]:
def run_rclone_command(command):
    try:
        # Run command
        result = subprocess.run(command, shell=True, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        # Check if the command was executed successfully
        if result.returncode == 0:
            print("Command executed successfully!")
            print("Output:", result.stdout)
        else:
            print("Error in command execution")
            print("Error:", result.stderr)
    except subprocess.CalledProcessError as e:
        print("An error occurred while trying to execute the command.")
        print(e)

# BASE_DIR = '/public/home/zecheng/workspace/zecheng'
# BASE_DIR = '/nvme/zecheng'
BASE_DIR = '/vepfs/wcf/G/zecheng'
# LOCAL_CKPT_DIR = '/vepfs/wcf/G/zecheng/ckpt/llama3_post_train_langauge_modeling'
# REMOTE_CKPT_DIR = '/ckpt/llama3_post_train_langauge_modeling'
# LOCAL_CKPT_DIR = '/public/home/zecheng/workspace/zecheng/ckpt/llama3_instruct_tuning'
LOCAL_CKPT_DIR = f'{BASE_DIR}/ckpt/fix_lcao_chunk_16_size_1024'
REMOTE_CKPT_DIR = '/' + '/'.join(LOCAL_CKPT_DIR.split('/')[-2:])

# UPLOAD_TEMPLATE_CONFIG = 'rclone copy {ckpt_dir}/checkpoint-{ckpt_num}/adapter_config.json lijt1:{remote_dir}/checkpoint-{ckpt_num}/'
# UPLOAD_TEMPLATE_MODEL = 'rclone copy {ckpt_dir}/checkpoint-{ckpt_num}/adapter_model.safetensors lijt1:{remote_dir}/checkpoint-{ckpt_num}/'

DOWNLOAD_TEMPLATE_CONFIG = 'rclone copy lijt1:{remote_dir}/checkpoint-{ckpt_num}/adapter_config.json {ckpt_dir}/checkpoint-{ckpt_num}/'
DOWNLOAD_TEMPLATE_MODEL = 'rclone copy lijt1:{remote_dir}/checkpoint-{ckpt_num}/adapter_model.safetensors {ckpt_dir}/checkpoint-{ckpt_num}/'

print('local:', LOCAL_CKPT_DIR)
print('remote:', REMOTE_CKPT_DIR)

# print(DOWNLOAD_TEMPLATE_CONFIG.format(ckpt_dir=LOCAL_CKPT_DIR, ckpt_num=50, remote_dir=REMOTE_CKPT_DIR))

for i in trange(150, 601, 50):
    run_rclone_command(DOWNLOAD_TEMPLATE_CONFIG.format(ckpt_dir=LOCAL_CKPT_DIR, ckpt_num=i, remote_dir=REMOTE_CKPT_DIR))
    run_rclone_command(DOWNLOAD_TEMPLATE_MODEL.format(ckpt_dir=LOCAL_CKPT_DIR, ckpt_num=i, remote_dir=REMOTE_CKPT_DIR))
    # run_rclone_command(DOWNLOAD_TEMPLATE_CONFIG.format(ckpt_dir=LOCAL_CKPT_DIR, ckpt_num=i, remote_dir=REMOTE_CKPT_DIR))
    # run_rclone_command(DOWNLOAD_TEMPLATE_MODEL.format(ckpt_dir=LOCAL_CKPT_DIR, ckpt_num=i, remote_dir=REMOTE_CKPT_DIR))

local: /vepfs/wcf/G/zecheng/ckpt/fix_lcao_chunk_16_size_1024
remote: /ckpt/fix_lcao_chunk_16_size_1024


  0%|          | 0/10 [00:00<?, ?it/s]

Command executed successfully!
Output: 


 10%|█         | 1/10 [00:01<00:13,  1.52s/it]

Command executed successfully!
Output: 
Command executed successfully!
Output: 


...

Command executed successfully!
Output: 
Command executed successfully!
Output: 


 90%|█████████ | 9/10 [00:25<00:03,  3.67s/it]

Command executed successfully!
Output: 
Command executed successfully!
Output: 


100%|██████████| 10/10 [00:28<00:00,  2.85s/it]

Command executed successfully!
Output: 


In [4]:
data = datasets.load_from_disk('/nvme/zecheng/data/iclr2025/llama3-80k-train-data/long-llm-score/chunk_16_size_1024/gpt-bio_book')
data[0].keys()

dict_keys(['all_ref_text', 'combined_question', 'final_answer', 'label', 'prefix_a', 'siffix_a', 'judge_scores', 'judger_preds'])

In [2]:
tmp = torch.load('/data/zecheng/ckpt/lcao_v6/adapter_model.bin')

In [3]:
tmp

OrderedDict([('base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight',
              tensor([[ 0.0101, -0.0045, -0.0085,  ...,  0.0025, -0.0221, -0.0051],
                      [-0.0215,  0.0074,  0.0110,  ..., -0.0135, -0.0042,  0.0103],
                      [-0.0156,  0.0102,  0.0145,  ..., -0.0092,  0.0108, -0.0044],
                      ...,
                      [ 0.0050, -0.0072,  0.0078,  ...,  0.0153,  0.0065,  0.0085],
                      [ 0.0095, -0.0181, -0.0158,  ...,  0.0017, -0.0197, -0.0105],
                      [ 0.0171,  0.0106, -0.0046,  ..., -0.0062, -0.0060, -0.0130]],
                     requires_grad=True)),
             ('base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight',
              tensor([[-4.9228e-03,  9.9637e-03, -5.2531e-03,  ..., -5.4938e-03,
                        8.5742e-04, -8.6134e-03],
                      [-2.8450e-03,  4.1072e-03,  1.8565e-03,  ..., -2.0054e-03,
                       -2.8104e-03, -